<a href="https://colab.research.google.com/github/sameh00alsharawy/machine-learning-/blob/master/Dog_breed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget --no-check-certificate \
    http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar \
    -O /tmp/cats_and_dogs_filtered.zip

--2020-08-20 22:06:46--  http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
Resolving vision.stanford.edu (vision.stanford.edu)... 171.64.68.10
Connecting to vision.stanford.edu (vision.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 793579520 (757M) [application/x-tar]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>] 756.82M  97.7MB/s    in 8.5s    

2020-08-20 22:06:54 (88.7 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [793579520/793579520]



In [ ]:
import os
import tarfile

local_tar = '/tmp/cats_and_dogs_filtered.zip'
tar = tarfile.open(local_tar)
tar.extractall()
tar.close()


In [ ]:
from keras.preprocessing.image import load_img
# load the image
img = load_img('/content/Images/n02085620-Chihuahua/n02085620_10074.jpg')
print(type(img))
print(img.format)
print(img.mode)
print(img.size)
img.show()

<class 'PIL.JpegImagePlugin.JpegImageFile'>
JPEG
RGB
(333, 500)


In [ ]:
import numpy as np
import pandas as pd 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.applications import ResNet50

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics

In [ ]:
model = keras.Sequential()
model.add(ResNet50(include_top=False, weights='imagenet', pooling='avg', ))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(1024, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(512, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(120, activation='softmax'))

model.layers[0].trainable = True

model.compile(optimizer = 'adam', loss = keras.losses.sparse_categorical_crossentropy, metrics = ['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
batch_normalization_3 (Batch (None, 2048)              8192      
_________________________________________________________________
dropout_3 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_4 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)              

In [ ]:
model.layers

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/content/Images'
img_height = 224
img_width = 224
batch_size=128
train_datagen = ImageDataGenerator(rescale=1./255,
                                   validation_split=0.2) 

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation') # set as validation data


Found 16508 images belonging to 120 classes.
Found 4072 images belonging to 120 classes.


In [ ]:
hist=model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 20    
)


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
128/128 [==============================] - 109s 854ms/step - loss: 3.5112 - accuracy: 0.1905 - val_loss: 5.0590 - val_accuracy: 0.0096
Epoch 2/20
128/128 [==============================] - 106s 827ms/step - loss: 2.1080 - accuracy: 0.4168 - val_loss: 5.6737 - val_accuracy: 0.0096
Epoch 3/20
128/128 [==============================] - 106s 826ms/step - loss: 1.5219 - accuracy: 0.5493 - val_loss: 9.5932 - val_accuracy: 0.0096
Epoch 4/20
128/128 [==============================] - 107s 837ms/step - loss: 1.1877 - accuracy: 0.6427 - val_loss: 6.0558 - val_accuracy: 0.0126
Epoch 5/20
128/128 [==============================] - 107s 836ms/step - loss: 0.8746 - accuracy: 0.7291 - val_loss: 5.9074 - val_accuracy: 0.0111
Epoch 6/20
 45/128 [=========>....................] - ETA: 58s - loss: 0.6183 - accuracy: 0.8031

KeyboardInterrupt: ignored

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
loss, acc=model.evaluate_generator(validation_generator)

In [ ]:
print('model loss : {}'.format(loss))
print('model acc  : {}%'.format(acc*100))